<center><img src="http://i.imgur.com/sSaOozN.png" width="500"></center>

## Course: Computational Thinking for Governance Analytics

### Prof. José Manuel Magallanes, PhD 
* Visiting Professor of Computational Policy at Evans School of Public Policy and Governance, and eScience Institute Senior Data Science Fellow, University of Washington.
* Professor of Government and Political Methodology, Pontificia Universidad Católica del Perú. 

_____
<a id='home'></a>

# Data Preprocessing in Python: Data gathering

0. [The GitHub Repo](#part0) 
1. [Uploading files](#part1a)
2. [APIs](#part2) 
3. [Scraping](#part3) 
4. [Social media](#part4) 

____
<a id='part0'></a>

## 0. The GitHub Repo

One the first steps you should take when preparing an analytics project is to know where to read your files from. One usually gets a file from an email, or finds it in website, and just download it to the local computer. 
However, different from most software, like SPSS, EXCEL and STATA, **Python** or **R** need to know the **path** to the file precisely. Sometimes you are not aware of the folders' **tree** in your computer. For instance, once I have opened this Jupyter Notebook, I can find its path like this:

In [1]:
import os
os.getcwd() # this is a comment, Python will not pay attention to this.

'/Users/Varsha/GitHub/comp-thinking'

You may not see the same path as the one above. It depends in at least a couple of things, the location of the folder you created in your computer, and the **operating system** you are using (i.e. Windows, OSX, Linux).

There is nothing wrong in using the files in a folder in your computer, but I recommend creating a *repo*sitory in Github for your _CTGA_ course, and synchronize that repo with your computer folder. Let's follow these steps:

1. Go to [github.com](https://github.com/), and sign up.
2. Install the Githib desktop app in your computer. It is available [here](https://desktop.github.com/). Sign in to the app.
3. Once you are signed in the Github web, create a repository. Choose a name. **DO NOT forget** to select the option to add a READ ME file, choose a **LICENSE** too (I recommend MIT).
4. **Clone** the repository created into your computer. You can select where to created.
5. Go to this [link](https://drive.google.com/drive/folders/1nfr3eHiTQVg7rcgVvXaxAbjdo0dCLqjn?usp=sharing) and download all the files you see there in the folder with the Github repository you just created.
6. **Push** the files in the local folder to your Github repo ('the cloud').

Once your local folder and the cloud are synchronized, you may go to the cloud repo and find the link to the data. This requires that you click on the name of the file in the repo, and get the link in the **download** or **raw** option; **DO NOT** use the link you see on the URL.

[home](#home)

____
<a id='part1a'></a>

## 1. Uploading 'proprietary software' files

Several times, you may find that you are given a file that was previously prepared with proprietary software. The most common in the policy field are:

* SPSS (file extension: **sav**).
* STATA (file extension: **dta**).
* EXCEL (file extension: **xlsx** or **xls**).

Getting these files up and running is the easiest, as they are often well organized and do not bring much pre processing challenges. They generally have the data organized in tables, where rows are the cases and columns are the variables. This structure is known by the data science community as the **data frame**. This data structure is not native to Python (it is in R), so when using Python we will need to use **Pandas**.

Let me use the links I have from my own repo, but this is the perfect time for you to check if Python can access your files from the cloud. Let's save the file locations first:

In [ ]:
cloudLocation_AllMyFiles='https://github.com/EvansDataScience/data/raw/master/'

linkToSTATA_File=cloudLocation_AllMyFiles+'hsb_ok.dta' # '+' concatenates text.
linkToSPSS_File=cloudLocation_AllMyFiles+'hsb_ok.sav'
linkToEXCEL_File=cloudLocation_AllMyFiles+'hsb_ok.xlsx'

Now, make sure you have **Pandas** installed:

In [ ]:
!pip show pandas

If you do **not have** Pandas, you can also install it using the Anaconda Navigator or via the Terminal. This code can install it from Jupyter notebook:

In [ ]:
#!pip install pandas   

Since I have Pandas, I wrote a **_#_** before the code above. Please so the same every time you have code you do not need to execute more than once.
 
Notice that the **show** command also informed the version I have. If you have an older version of Pandas than the one I have, you can update yours like this:

In [ ]:
#!pip install pandas -U

____

### 1.1 Reading STATA and EXCEL:

Most files are easy to read in. In this case, STATA will give no extra work, but reading in Excel file requires the libraris **xlrd** (for xls) or the library **openpyxl** (for xlsx), just check if you have them; otherwise, install them using **!pip install**. 

In [ ]:
import pandas as pd # activating pandas

dfStata=pd.read_stata(linkToSTATA_File)
dfExcel=pd.read_excel(linkToEXCEL_File,usecols='B:P') #this omits first column (row number from Excel).

#you can also use:

# import pandas
# dfStata=pandas.read_stata(linkToSTATA_File)
# dfExcel=pandas.read_excel(linkToEXCEL_File)


Above, you are telling Python to use a function from Pandas. Generally, this input functions have a reciprocal output function (for example: write_stata). 

The objects **dfStata** and **dfExcel** are data frames:

In [ ]:
type(dfStata) , type(dfExcel)

They should have the same information:

In [ ]:
dfStata.shape, dfExcel.shape

You can check some rows from both:

In [ ]:
dfStata.head()

In [ ]:
dfExcel.tail()
#?pd.read_excel

Remember that an Excel file can have **several sheets**, so you need to call each by its location (number) or name (label):

In [ ]:
fileXLSXs=cloudLocation_AllMyFiles+'WA_COVID19_Cases_Hospitalizations_Deaths.xlsx'

#opening each in a different data frame.
dataExcelCovidCases=pd.read_excel(fileXLSXs,sheet_name='Cases')
dataExcelCovidDeaths=pd.read_excel(fileXLSXs,sheet_name='Deaths') 
dataExcelCovidHospitalizations=pd.read_excel(fileXLSXs,sheet_name='Hospitalizations') 
dataExcelCovidDataDictionary=pd.read_excel(fileXLSXs,sheet_name='Data Dictionary') 

Let's check one of the dataframes:

In [ ]:
dataExcelCovidDataDictionary

If you need some more space to see cell contents:

In [ ]:
# if you need more space:
pd.set_option('max_colwidth', 800)
dataExcelCovidDataDictionary

[home](#home)

____

### 1.2 Reading an SPSS file

Pandas has a **read_spss** function. But, before using it, verify you have the library **[pyreadstat](https://github.com/Roche/pyreadstat)** installed.

In [ ]:
!pip show pyreadstat 

Notice that **pyreadstat** cannot read from an url, so we need some some extra coding:

In [ ]:
# to open link:
from urllib.request import urlopen

response = urlopen(linkToSPSS_File) 

# reading contents
content = response.read() 

# opening a file new file locally
fhandle = open('savingLocally.sav', 'wb') #wb?

# saving contents in that file
fhandle.write(content) 

# closing the file
fhandle.close() 

If you see the folder where you are writing this code, you should find that the file from the url is now in your computer. Now just read it:

In [ ]:
dataSPSS= pd.read_spss("savingLocally.sav")

# here it is
dataSPSS

[home](#home)

____

<a id='part2'></a>

## 2. Collecting data from APIs

Open data portals from the government and other organizations have APIs, a service that allows you to collect their data. Let's take a look a Seattle data about [Seattle Real Time Fire 911 Calls](https://dev.socrata.com/foundry/data.seattle.gov/kzjm-xkqj):

In [ ]:
from IPython.display import IFrame  
fromAPI="https://dev.socrata.com/foundry/data.seattle.gov/kzjm-xkqj" 
IFrame(fromAPI, width=900, height=500)

That page tells you how to get the data into pandas. But first, you need to install **sodapy**. Then you can continue:

In [ ]:
#!pip install sodapy

Let's follow some steps, according to the API:

In [ ]:
from sodapy import Socrata

# Unauthenticated client (using 'None')

client = Socrata("data.seattle.gov", None)

# If you have credentials:
# client = Socrata(data.seattle.gov,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 500 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("kzjm-xkqj", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

You can see the results now:

In [ ]:
results_df

Data from APIs may need some more pre processing than the previous files. Besides, you should study the API documentation to know how to interact with the portal. Not every open data portal behaves the same.

[home](#home)
____
<a id='part3'></a>

## 3. Scraping

Sometimes you are interested in data from the web. Let me get a table from wikipedia:

In [ ]:
from IPython.display import IFrame  
wikiLink="https://en.wikipedia.org/wiki/Democracy_Index" 
IFrame(wikiLink, width=700, height=300)

I will use pandas to get the table, but you need to install these first:
* html5lib 
* beautifulsoup
* lxml

In [ ]:
dataWIKI=pd.read_html(wikiLink,header=0,flavor='bs4',attrs={'class': 'wikitable'})

Pandas has this command **read_html** that will save lots of coding, above I just said:
* The link to the webpage.
* The position of the header.
* The external library that will be used to extract the text (_flavor_).
* The attributes of the table.

dataWIKI is not a data frame:

In [ ]:
type(dataWIKI)

The command **read_html** returns all the elements from the link with the same attributes. Let's see how many there are:

In [ ]:
len(dataWIKI)

This means you have six tables. Ours is the first one?

In [ ]:
# remember that Python starts counting in ZERO!
dataWIKI[0]

It is the 6th table:

In [ ]:
dataWIKI[5]

Tables scrapped will bring different cleaning challenges. 

[home](#home)
____
<a id='part4'></a>

## Social media data

Social media offer APIs too that allow you to get _some_ data. In general, you need to register as a developer. Once you are a confirmed developer, Twitter, Facebook and others will allow you to get _some_ of their data (the more you pay the more they offer). 

Let's pay attention to Twitter. First, install **tweepy**

In [ ]:
#!pip install tweepy

Tweepy is the key library, but you may need several other libraries according to your goals.

In [ ]:
import tweepy

Let me introduce myself to Twitter:

In [ ]:
# credentials
consumer_key = 'XXX'
consumer_secret = 'YYY'
access_token =  'ZZZ'
access_token_secret = 'AAA'

# introducing myself:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api=tweepy.API(auth, wait_on_rate_limit=True,timeout=60,
               parser=tweepy.parsers.JSONParser())

Let me ask for some tweets from a particular user:

In [ ]:

who='@Minsa_Peru'
howMany=20
gottenTweets = api.user_timeline(screen_name = who, 
                                 count = howMany, 
                                 include_rts = False,
                                 tweet_mode="extended")

In the previous cases, I got a table (a data frame), you should always check what you have:

In [ ]:
type(gottenTweets)

I have a list, then I could ask how many tweets I got (just to confirm):

In [ ]:
len(gottenTweets)

Let me view what I have in the first tweet:

In [ ]:
gottenTweets[0]

It will take some time to become familiar with a [tweet object structure](https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet). Let's find out how the tweets are currently stored:

In [ ]:
type(gottenTweets[0])

Now you know that each tweet is stored as a dictionaary. Let me see the dict **keys**:

In [ ]:
gottenTweets[0].keys()

Let me recover some info from each tweet:

In [ ]:
dates=[t['created_at'] for t in gottenTweets]
ids=[t['id'] for t in gottenTweets]
rts=[t['retweet_count'] for t in gottenTweets]
likes=[t['favorite_count'] for t in gottenTweets]
text=[t['full_text'] for t in gottenTweets]

Each of the objects created is a list (dates, ids,rts,likes and text). Let me show you one:

In [ ]:
text

Let me create a data frame with those lists:

In [ ]:
tweetsAsDF=pd.DataFrame({'dates':dates,'ids':ids,'rts':rts,'likes':likes,'text':text})

In [ ]:
tweetsAsDF

[home](#home)